This file will run thresholding memory experiments on the rotated surface code, using MWPM and BPOSD, under depolarizing noise. 

In [7]:
# Import necessary objects
from dotg.circuits import SurfaceCode
from dotg.noise import DepolarizingNoise
from dotg.decoders import (
    MinimumWeightPerfectMatching,
    BPOSD,
    LDPCDecoderOptions,
    OSDMethods,
)

In [8]:
# Define parameters
from tqdm import tqdm
import numpy as np

# Ambitious!
NUM_SHOTS = 1e6

# Decent number of distances for more accurate thresholding
DISTANCES = [5, 7, 9, 11, 13]

# We expect the threshold to lie between 0.1% and 1% physical
PHYSICAL_ERRORS = np.logspace(-3, -2, 20)

In [9]:
# Function to quickly create noisy circuits
def noisy_circuit(distance: int, physical_error: float):
    noise_model = DepolarizingNoise(physical_error=physical_error)
    circuit = SurfaceCode.Rotated(distance=distance).memory
    return noise_model.permute_circuit(circuit=circuit)

In [10]:
# Experiment cell
logical_error_mwpm = {}
logical_error_bposd = {}

for distance in tqdm(DISTANCES):
    logical_error_curve = []
    for physical in PHYSICAL_ERRORS:
        # Restrict shot number for high physical errors
        num_shots = int(min(NUM_SHOTS, 1 / physical**2))

        circ = noisy_circuit(distance=distance, physical_error=physical)

        mwpm = MinimumWeightPerfectMatching(circuit=circ)
        logical_mwpm = mwpm.logical_error(num_shots=num_shots)

        bposd = BPOSD(
            circuit=circ,
            decoder_options=LDPCDecoderOptions(
                max_iterations=distance, osd_method=OSDMethods.EXHAUSTIVE
            ),
        )
        logical_bposd = bposd.logical_error(num_shots=num_shots)

        logical_error_curve = [(logical_mwpm, logical_bposd)]
    
    mwpm_curve, bposd_curve = zip(*logical_error_curve)

    logical_error_mwpm[distance] = mwpm_curve
    logical_error_bposd[distance] = bposd_curve

  0%|          | 0/5 [00:00<?, ?it/s]


TypeError: 'Rotated' object is not iterable